In [50]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from pathlib import Path
import re
import pickle
# ver qtos o modelo apenas de ner acertaria
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize 
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json   
from importlib import reload  # Python 3.4+
import random
import model as mod
from model import BertForChunkClassification
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

from torch.utils.data import DataLoader
from importlib import reload 
#from eval import predict
import eval
#import importlib
#importlib.reload(module)
import dataset
from dataset import InputFeatures, load_and_cache_examples
import functionsAval as f

In [51]:
f = reload(f)
reload(dataset)
reload(eval)
reload(mod)

# em numero de frases
#BATCH=30
BATCH=100 
#BATCH=800
#BATCH=8000 
print('BATCH:', BATCH)

BATCH: 100


In [24]:
# em numero de frases
#BATCH=30
BATCH=100 
#BATCH=800
#BATCH=8000 
print('BATCH:', BATCH)

dicSentences_new_test = f.loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
#print(dicSentences_new_test[0])
print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))


sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        tokens = [tok[0] for tok in tokens]
        sentences.append(' '.join(tokens).strip())
#print(sentences[0])

tags, tokens = f.predictBERTNER_IO(sentences, 'all')
dic_predictions = f.getDicPredictions(tags, tokens)
print(dic_predictions[0])
print('len(dic_predictions):', len(dic_predictions))
print(dic_predictions[9])
f.save_obj('dic_predictions_results_ner_'+str(BATCH), dic_predictions)
#dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))
print('verificando dados:')
for key, value in dic_predictions.items():
    print('key:',key)
    print(dic_predictions[key])
    if key>2:
        break
        
print('len(dicSentences_new_test):', len(dicSentences_new_test))
print('len(dic_predictions):', len(dic_predictions))

region_true_list, region_pred_list, lista_erros = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, dic_predictions)
f.save_obj('region_true_list'+str(BATCH), region_true_list)
print('region_pred_list[:4]:', region_pred_list[:4])
print('region_true_list[:4]:', region_true_list[:4])
print('lista_erros[:8]:', lista_erros[:8])
print('len(lista_erros):', len(lista_erros))
print('len(set(lista_erros)):', len(set(lista_erros)))
#print(dic_predictions[8])
#print(dicSentences_new_test[8][1])
print('len(region_true_list):', len(region_true_list))
print('len(region_pred_list):', len(region_pred_list))
#print('pred:',region_pred_list[:15])
#print('true:',region_true_list[:15])

print('-----Avaliando só modelo de NER:-----')

print(classification_report(region_true_list, region_pred_list, digits=6))
print(confusion_matrix(region_true_list, region_pred_list))


BATCH: 100
Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Nega', 0, 422], ['sincope', 1, 427], ['.', 2, 434]], [['sincope', [1], 'Problema']]]
numero de sentencas no total: 100
idx2tag: {0: 'Teste', 1: 'Anatomia', 2: 'O', 3: 'Problema', 4: 'Tratamento', 5: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 100
[[['Abd', 0], ['globoso', 1], [',', 2], ['flacido', 3], [',', 4], ['indolor', 5], ['a', 6], ['palpacao', 7], [',', 8], ['sem', 9], ['VCM', 10], ['.', 11]], [['Abd', [0], 'Anatomia'], ['VCM', [10], 'Problema']]]
verificando dados:
key: 0
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
key: 1
[[['Em', 0], ['acompanhamento', 1], ['no', 2], ['ambualtorio', 3], ['há', 4], ['5', 5], ['anos', 6], ['por', 7], ['FA', 8], [',', 9], ['uso', 10], ['de', 11], ['marevan', 12], ['5mg', 13], ['1', 14], ['x', 15], ['ao', 16], ['dia', 17], ['.', 18]], [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]

### Agora, modelo de SpanClassification

In [70]:
print('-----Avaliando só com modelo de Spam:-----')
model = f.getModel()
dic_predictions =  f.load_obj('dic_predictions_results_ner_'+str(BATCH)) # TODO
dic_sentencesTrainDev = f.load_obj('dic_sentencesTrainDev')
dicPosTagger, _ = f.getDicPosTagger(dic_sentencesTrainDev)
lista_postaggers_entidades = f.getListaPostaggerEntidades(dic_predictions, dicPosTagger)
# para gerar arquivo de predicoes (com as tags <e1>)
print('Primeiro, com filtro postagger:')
combinacaoEntidadesAll, pullando_termos_pos_tagger = f.getCombinacaoEntidades(dic_predictions, True, dicPosTagger, lista_postaggers_entidades)
f.save_obj('combinacaoEntidadesAll_comPostagger_'+str(BATCH), combinacaoEntidadesAll)
f.gravaArquivoPredict(combinacaoEntidadesAll, tipo='io', postagger=True)
print('Alguns termos pulados:')
print(pullando_termos_pos_tagger[:5])
# chamar predict
print('Chamando predict')
pred_region_labels = f.predictSpan(model)
print('len(pred_region_labels):', len(pred_region_labels))
print(region_true_list[:10])
print(pred_region_labels[:10])
#combinacaoEntidadesAll = load_obj(combinacaoEntidadesAll_'+str(BATCH))

# combinacaoEntidadesAll_pred = predicoes do SpanClassification no formato para comparar com gabarito
combinacaoEntidadesAll_pred = f.getCombinacaoEntidadesAll_pred(combinacaoEntidadesAll, pred_region_labels)
print('vai chamar getDicPredictionsSpan')
dic_predictions_span = f.getDicPredictionsSpan(combinacaoEntidadesAll_pred, dic_predictions)
f.save_obj('dic_predictions_span_com_postagger_'+str(BATCH), dic_predictions_span)
print('vai chamar AvalFinal')
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_span, BATCH)

-----Avaliando só com modelo de Spam:-----
loading model from C:\Users\lisat\OneDrive\jupyter notebook\span-model\model-exp3
num_labels: 5
Primeiro, com filtro postagger:
Com filtro-postagger
len(combinacaoEntidadesAll:) 100
len(pulando_termos_postagger): 233
numTotalEntidades: 585
numFrases: 86
gravando em: data/span.compostagger.predict
Alguns termos pulados:
[[['25'], '-NUM'], [['hipertrofia', 'concentrica', 'de'], '-N-N-PREP'], [['concentrica', 'de'], '-N-PREP'], [['concentrica', 'de', 'grau', 'discreto'], '-N-PREP-N-ADJ'], [['de', 'grau'], '-PREP-N']]
Chamando predict
['O', 'Problema', 'Tratamento', 'Teste', 'Anatomia']
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
LOOKING AT ./data\span.predict
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
*** Example ***
tokens: ['[CLS]', 'Em', 'ac', '##om', '##pan', '##ham', '##ento', 'no',

In [72]:
combinacaoEntidadesAll_pred[:10]

[[],
 ['Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .',
  [['Em acompanhamento no ambualtorio há 5 anos por <e1> FA </e1> , uso de marevan 5mg 1 x ao dia .',
    'O'],
   ['Em acompanhamento no ambualtorio há 5 anos por FA , uso de <e1> marevan 5mg </e1> 1 x ao dia .',
    'Tratamento'],
   ['Em acompanhamento no ambualtorio há 5 anos por FA , uso de <e1> marevan </e1> 5mg 1 x ao dia .',
    'Tratamento'],
   ['Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan <e1> 5mg </e1> 1 x ao dia .',
    'O']]],
 ['Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .',
  [['<e1> Comorbidades </e1> : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .',
    'O'],
   ['Comorbidades : <e1> DM </e1> há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicaz

In [136]:
dic_predictions_span[23]

[[['#', 0],
  ['Lucas', 1],
  [',', 2],
  ['78', 3],
  ['anos', 4],
  ['*', 5],
  ['HAS', 6],
  [',', 7],
  ['*', 8],
  ['Gota', 9],
  [',', 10],
  ['*', 11],
  ['DAC', 12],
  ['-', 13],
  ['IAM', 14],
  ['+', 15],
  ['ATC', 16],
  ['em', 17],
  ['2012', 18],
  ['(', 19],
  ['angina', 20],
  ['instavel', 21],
  [')', 22],
  ['-', 23],
  ['Ecocardiograma', 24],
  ['mar', 25],
  ['/', 26],
  ['14', 27],
  [':', 28],
  ['FE', 29],
  ['42', 30],
  ['%', 31],
  ['/', 32],
  ['comprometimento', 33],
  ['difuso', 34],
  ['do', 35],
  ['VE', 36],
  ['grau', 37],
  ['moderado', 38],
  [',', 39],
  ['insuf', 40],
  ['Mi', 41],
  ['discreta', 42],
  [',', 43],
  ['insuf', 44],
  ['Tri', 45],
  ['discreta', 46],
  [',', 47],
  ['hipetensao', 48],
  ['pulmonar', 49],
  ['discreta', 50],
  ['.', 51]],
 [['HAS', [6], 'Problema'],
  ['Gota', [9], 'Problema'],
  ['DAC', [12], 'Problema'],
  ['IAM', [14], 'Problema'],
  ['ATC', [16], 'Tratamento'],
  ['angina instavel', [20, 21], 'Problema'],
  ['Ecocar

In [75]:
# para gerar arquivo de predicoes (com as tags <e1>)
print('Agora, sem filtro postagger:')
combinacaoEntidadesAll, pullando_termos_pos_tagger = f.getCombinacaoEntidades(dic_predictions, False, '', '')
f.save_obj('combinacaoEntidadesAll_comPostagger_'+str(BATCH), combinacaoEntidadesAll)
f.gravaArquivoPredict(combinacaoEntidadesAll, tipo='io', postagger=False)
# chamar predict
print('Chamando predict')
pred_region_labels = f.predictSpan(model)
print('len(pred_region_labels):', len(pred_region_labels))
print(region_true_list[:10])
print(pred_region_labels[:50])
#combinacaoEntidadesAll = load_obj(combinacaoEntidadesAll_'+str(BATCH))

# combinacaoEntidadesAll_pred = predicoes do SpanClassification no formato para comparar com gabarito
combinacaoEntidadesAll_pred = f.getCombinacaoEntidadesAll_pred(combinacaoEntidadesAll, pred_region_labels)

dic_predictions_span = f.getDicPredictionsSpan(combinacaoEntidadesAll_pred, dic_predictions)
f.save_obj('dic_predictions_span_com_postagger_'+str(BATCH), dic_predictions_span)

region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_span, BATCH)

Agora, sem filtro postagger:
Sem filtro-postagger
len(combinacaoEntidadesAll:) 100
len(pulando_termos_postagger): 0
numTotalEntidades: 731
numFrases: 86
gravando em: data/span.sempostagger.predict
Chamando predict
['O', 'Problema', 'Tratamento', 'Teste', 'Anatomia']
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
LOOKING AT ./data\span.predict
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
*** Example ***
tokens: ['[CLS]', 'Em', 'ac', '##om', '##pan', '##ham', '##ento', 'no', 'amb', '##ual', '##torio', 'há', '5', 'anos', 'por', 'FA', ',', 'uso', 'de', 'mare', '##van', '5', '##m', '##g', '1', 'x', 'ao', 'dia', '.', '[SEP]']
lista_label: [1, 2, 0, 0]
indices_e1_sentenca: [[15, 15], [19, 23], [19, 20], [21, 23]]
*** Example ***
tokens: ['[CLS]', 'Como', '##rb', '##idades', ':', 'DM', 'há', '10', 'anos', 'em', 'uso', 'de', 'met', '##form'

In [76]:
dicSentences_new_test

{0: [[['Lucas', 0, 43],
   [',', 1, 48],
   ['74', 2, 50],
   ['anos', 3, 53],
   ['.', 4, 57]],
  []],
 1: [[['Em', 0, 59],
   ['acompanhamento', 1, 62],
   ['no', 2, 77],
   ['ambualtorio', 3, 80],
   ['há', 4, 92],
   ['5', 5, 95],
   ['anos', 6, 97],
   ['por', 7, 102],
   ['FA', 8, 106],
   [',', 9, 108],
   ['uso', 10, 110],
   ['de', 11, 114],
   ['marevan', 12, 117],
   ['5mg', 13, 125],
   ['1', 14, 129],
   ['x', 15, 131],
   ['ao', 16, 133],
   ['dia', 17, 136],
   ['.', 18, 139]],
  [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]],
 2: [[['Comorbidades', 0, 142],
   [':', 1, 154],
   ['DM', 2, 156],
   ['há', 3, 159],
   ['10', 4, 162],
   ['anos', 5, 165],
   ['em', 6, 170],
   ['uso', 7, 173],
   ['de', 8, 177],
   ['metformina', 9, 180],
   ['850mg', 10, 191],
   ['3', 11, 197],
   ['cp', 12, 199],
   ['/', 13, 201],
   ['dia', 14, 202],
   [',', 15, 205],
   ['acarbose', 16, 207],
   ['1', 17, 216],
   ['cp', 18, 218],
   ['/', 19, 220],
   ['dia', 20

In [77]:
dic_predictions

{0: [[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []],
 1: [[['Em', 0],
   ['acompanhamento', 1],
   ['no', 2],
   ['ambualtorio', 3],
   ['há', 4],
   ['5', 5],
   ['anos', 6],
   ['por', 7],
   ['FA', 8],
   [',', 9],
   ['uso', 10],
   ['de', 11],
   ['marevan', 12],
   ['5mg', 13],
   ['1', 14],
   ['x', 15],
   ['ao', 16],
   ['dia', 17],
   ['.', 18]],
  [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]],
 2: [[['Comorbidades', 0],
   [':', 1],
   ['DM', 2],
   ['há', 3],
   ['10', 4],
   ['anos', 5],
   ['em', 6],
   ['uso', 7],
   ['de', 8],
   ['metformina', 9],
   ['850mg', 10],
   ['3', 11],
   ['cp', 12],
   ['/', 13],
   ['dia', 14],
   [',', 15],
   ['acarbose', 16],
   ['1', 17],
   ['cp', 18],
   ['/', 19],
   ['dia', 20],
   ['e', 21],
   ['glicazida', 22],
   ['60mg', 23],
   ['2', 24],
   ['cp', 25],
   ['/', 26],
   ['dia', 27],
   ['e', 28],
   ['insulina', 29],
   ['(', 30],
   ['24', 31],
   ['-', 32],
   ['0', 33],
   ['-', 34],
 

In [78]:
dic_predictions_span

{0: [[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []],
 1: [[['Em', 0],
   ['acompanhamento', 1],
   ['no', 2],
   ['ambualtorio', 3],
   ['há', 4],
   ['5', 5],
   ['anos', 6],
   ['por', 7],
   ['FA', 8],
   [',', 9],
   ['uso', 10],
   ['de', 11],
   ['marevan', 12],
   ['5mg', 13],
   ['1', 14],
   ['x', 15],
   ['ao', 16],
   ['dia', 17],
   ['.', 18]],
  [['marevan 5mg', [12, 13], 'Tratamento'], ['marevan', [12], 'Tratamento']]],
 2: [[['Comorbidades', 0],
   [':', 1],
   ['DM', 2],
   ['há', 3],
   ['10', 4],
   ['anos', 5],
   ['em', 6],
   ['uso', 7],
   ['de', 8],
   ['metformina', 9],
   ['850mg', 10],
   ['3', 11],
   ['cp', 12],
   ['/', 13],
   ['dia', 14],
   [',', 15],
   ['acarbose', 16],
   ['1', 17],
   ['cp', 18],
   ['/', 19],
   ['dia', 20],
   ['e', 21],
   ['glicazida', 22],
   ['60mg', 23],
   ['2', 24],
   ['cp', 25],
   ['/', 26],
   ['dia', 27],
   ['e', 28],
   ['insulina', 29],
   ['(', 30],
   ['24', 31],
   ['-', 32],
   ['0', 33],
   ['-'

In [79]:
f = reload(f)
reload(dataset)
reload(eval)
reload(mod)


<module 'model' from 'C:\\Users\\lisat\\OneDrive\\jupyter notebook\\spanclassification\\avaliacao\\model.py'>

In [80]:
a=[[0],[1,2]]
[1,2] in a

True

In [35]:
dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))
#dic_predictions

In [81]:
# juntar all_predictions
print('juntando span + ner')
#dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))
#dic_predictions_all = f.getDicPredictionsAll(combinacaoEntidadesAll_pred, dic_predictions)

dic_predictions_all = {}
for key, value in dic_predictions.items():
    #dic_predictions_all[key] = value.copy()
    tokens = value[0].copy()
    listaEntidadesNer = value[1].copy()
    listaIndicesNer=list()
    for entidadeNer in listaEntidadesNer:
        listaIndicesNer.append(entidadeNer[1])
    #print('listaIndicesNer:', listaIndicesNer)
    listaEntidadesSpan = dic_predictions_span[key][1]
    for entidadeSpan in listaEntidadesSpan:
        indices = entidadeSpan[1]
        if indices in listaIndicesNer:
            #print('ja tem, pulando')
            pass
        else:
            #print('incluindo:', entidadeSpan)
            listaEntidadesNer.append(entidadeSpan)
    dic_predictions_all[key] = [tokens, listaEntidadesNer]
    #if key>5:
    #    break

    

juntando span + ner


In [91]:
dic_predictions_all[13]

[[['Ecocardiograma', 0],
  ['-', 1],
  ['ventrículo', 2],
  ['esquerdo', 3],
  ['com', 4],
  ['hipertrofia', 5],
  ['concentrica', 6],
  ['de', 7],
  ['grau', 8],
  ['discreto', 9],
  ['e', 10],
  ['função', 11],
  ['sistólica', 12],
  ['preservada', 13],
  ['.', 14]],
 [['Ecocardiograma', [0], 'Teste'],
  ['ventrículo esquerdo', [2, 3], 'Anatomia'],
  ['hipertrofia concentrica de grau discreto', [5, 6, 7, 8, 9], 'Problema'],
  ['ventrículo', [2], 'Anatomia'],
  ['hipertrofia', [5], 'Problema'],
  ['hipertrofia concentrica', [5, 6], 'Problema']]]

In [82]:
len(dic_predictions_span)

100

In [86]:
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH)

numErro1: 48
numErro2: 205
              precision    recall  f1-score   support

    Anatomia   0.516129  0.592593  0.551724        54
           O   0.000000  0.000000  0.000000       205
    Problema   0.450704  0.849558  0.588957       113
       Teste   0.707547  0.903614  0.793651        83
  Tratamento   0.611111  0.859375  0.714286        64

    accuracy                       0.497110       519
   macro avg   0.457098  0.641028  0.529724       519
weighted avg   0.340343  0.497110  0.400641       519



### Com pos-processamento

Regras:

1) primeiro, aceitar sempre do primeiro modelo (que pega as entidades de fora)
2) depois, pega os de dentro (aninhadas), apenas se o tipo for diferente do tipo da entidade de fora... se for do mesmo tipo, não pega...

In [113]:
def isEntidadeAninhada(a, b):
    inicio1=a[0]
    fim1=a[-1]
    #print(inicio1, fim1)
    inicio2=b[0]
    fim2=b[-1]
    #print(inicio2, fim2)
    if inicio1 >=inicio2 and fim1 <=fim2:
        return True
    else:
        return False
isEntidadeAninhada([0], [2, 3])

False

In [127]:
# juntar all_predictions
print('juntando span + ner, com pos processamento')
#dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))
#dic_predictions_all = f.getDicPredictionsAll(combinacaoEntidadesAll_pred, dic_predictions)

def juntaPredictons(dic_predictions, dic_predictions_span):
    dic_predictions_all = {}
    for key, value in dic_predictions.items():
        #dic_predictions_all[key] = value.copy()
        tokens = value[0].copy()
        listaEntidadesNer = value[1].copy()
        listaIndicesNer=list()
        listaTipoNer=list()
        for entidadeNer in listaEntidadesNer:
            listaIndicesNer.append(entidadeNer[1])
            listaTipoNer.append(entidadeNer[2])
        #print('listaIndicesNer:', listaIndicesNer)
        listaEntidadesSpan = dic_predictions_span[key][1]
        for entidadeSpan in listaEntidadesSpan:
            indices = entidadeSpan[1]
            if indices in listaIndicesNer:
                #print('ja tem, pulando')
                pass
            else:
                #print('incluindo:', entidadeSpan)
                # só acrescenta se nao tem entidade mais externa do mesmo tipo
                # ver pelo indice
                isAninhada=False
                tipo=entidadeSpan[2]
                for indicesNer, tipoNer in zip(listaIndicesNer, listaTipoNer):
                    isAninhada = isEntidadeAninhada(indices, indicesNer) # é aninhada com alguma entidade?
                    if isAninhada and tipoNer == tipo:
                        break
                # ver pelo tipo tb
                if not isAninhada:
                    print('Não é aninhada, incluindo:', entidadeSpan)
                    listaEntidadesNer.append(entidadeSpan)
                else:
                    #print('aninhada:', entidadeSpan)
                    pass
        dic_predictions_all[key] = [tokens, listaEntidadesNer]
        #if key>15:
        #    break
    return dic_predictions_all

dic_predictions_all = juntaPredictons(dic_predictions, dic_predictions_span)
#dic_predictions_all
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH)

juntando span + ner, com pos processamento
Não é aninhada, incluindo: ['glicazida', [22], 'Problema']
Não é aninhada, incluindo: ['comprometimento difuso do VE', [33, 34, 35, 36], 'Anatomia']
Não é aninhada, incluindo: ['do VE', [35, 36], 'Anatomia']
Não é aninhada, incluindo: ['do VE grau', [35, 36, 37], 'Anatomia']
Não é aninhada, incluindo: ['VE', [36], 'Anatomia']
Não é aninhada, incluindo: ['VE grau', [36, 37], 'Anatomia']
Não é aninhada, incluindo: ['VE grau moderado', [36, 37, 38], 'Anatomia']
Não é aninhada, incluindo: ['grau', [37], 'Anatomia']
Não é aninhada, incluindo: ['insuf Mi', [40, 41], 'Teste']
Não é aninhada, incluindo: ['discreta', [42], 'Teste']
Não é aninhada, incluindo: ['insuf Tri', [44, 45], 'Anatomia']
Não é aninhada, incluindo: ['Tri discreta', [45, 46], 'Teste']
Não é aninhada, incluindo: ['discreta', [46], 'Teste']
Não é aninhada, incluindo: ['em MMII', [4, 5], 'Anatomia']
Não é aninhada, incluindo: ['MMII', [5], 'Anatomia']
Não é aninhada, incluindo: ['tora

In [137]:
print('agora, com  filtro pos tagger')

#dic_predictions_span = f.load_obj('dic_predictions_span_com_postagger_'+str(BATCH))
dic_predictions_all = juntaPredictons(dic_predictions, dic_predictions_span)
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH)

agora, com  filtro pos tagger
Não é aninhada, incluindo: ['glicazida', [22], 'Problema']
Não é aninhada, incluindo: ['VE', [36], 'Anatomia']
Não é aninhada, incluindo: ['VE grau', [36, 37], 'Anatomia']
Não é aninhada, incluindo: ['grau', [37], 'Anatomia']
Não é aninhada, incluindo: ['insuf Mi', [40, 41], 'Teste']
Não é aninhada, incluindo: ['insuf Tri', [44, 45], 'Anatomia']
Não é aninhada, incluindo: ['tróficas', [27], 'Teste']
Não é aninhada, incluindo: ['repolarização', [43], 'Teste']
Não é aninhada, incluindo: ['mmii', [4], 'Anatomia']
numErro1: 74
numErro2: 45
              precision    recall  f1-score   support

    Anatomia   0.708333  0.314815  0.435897        54
           O   0.000000  0.000000  0.000000        45
    Problema   0.845455  0.823009  0.834081       113
       Teste   0.850575  0.891566  0.870588        83
  Tratamento   0.828125  0.828125  0.828125        64

    accuracy                       0.660167       359
   macro avg   0.646498  0.571503  0.593738     

In [134]:
dic_predictions_span = f.load_obj('dic_predictions_span_com_postagger_'+str(BATCH))

dic_predictions_span

{0: [[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []],
 1: [[['Em', 0],
   ['acompanhamento', 1],
   ['no', 2],
   ['ambualtorio', 3],
   ['há', 4],
   ['5', 5],
   ['anos', 6],
   ['por', 7],
   ['FA', 8],
   [',', 9],
   ['uso', 10],
   ['de', 11],
   ['marevan', 12],
   ['5mg', 13],
   ['1', 14],
   ['x', 15],
   ['ao', 16],
   ['dia', 17],
   ['.', 18]],
  [['marevan 5mg', [12, 13], 'Tratamento'], ['marevan', [12], 'Tratamento']]],
 2: [[['Comorbidades', 0],
   [':', 1],
   ['DM', 2],
   ['há', 3],
   ['10', 4],
   ['anos', 5],
   ['em', 6],
   ['uso', 7],
   ['de', 8],
   ['metformina', 9],
   ['850mg', 10],
   ['3', 11],
   ['cp', 12],
   ['/', 13],
   ['dia', 14],
   [',', 15],
   ['acarbose', 16],
   ['1', 17],
   ['cp', 18],
   ['/', 19],
   ['dia', 20],
   ['e', 21],
   ['glicazida', 22],
   ['60mg', 23],
   ['2', 24],
   ['cp', 25],
   ['/', 26],
   ['dia', 27],
   ['e', 28],
   ['insulina', 29],
   ['(', 30],
   ['24', 31],
   ['-', 32],
   ['0', 33],
   ['-'

dic_predictions_all

In [118]:
combinacaoEntidadesAll

[[],
 [['FA',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .',
   1,
   1],
  ['marevan 5mg',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .',
   2,
   2],
  ['marevan',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .',
   0,
   2]],
 [['Comorbidades',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .',
   1,
   1],
  ['DM',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .',
   1,
   1],
  ['metformina 850mg',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .',
   2,
   2],
  ['acarbose',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1